# Coriolis force

In [1]:
%matplotlib ipympl
def figure(name, nrows=1, ncols=1, *args, **kwargs):
    plt.close(name)
    return plt.subplots(nrows, ncols, num=name, *args, **kwargs)

import numpy as np
import pylab as plt
plt.style.use('default')

In [2]:
from ipywidgets import HBox, IntSlider, FloatSlider, FloatLogSlider, RadioButtons, Select
from scipy.integrate import solve_ivp
from mpl_toolkits.mplot3d import Axes3D
from numpy.linalg import norm
from numpy import cross
import scipy.constants as sc

### useful constants

In [3]:
g = 9.813
R0 = 6370 * 1e3
G = sc.G * 5.972 * 1e24 / R0**3
ka = 0.45 * np.pi * 0.05**2 * 1.341 / (2 * 0.5) 

In [4]:
def derivative1(t, y, w):
    dy = np.zeros(6)
    r = y[:3]
    v = y[3:]
    dy[:3] = y[3:]
    # we dont use z'' = -g, but r'' = -G*r!, it's way more general
    dy[3:] = -G * r - 2 * cross(w, v) - cross(w, cross(w, r))
    return dy

def derivative2(t, y, w):
    dy = np.zeros(6)
    r = y[:3]
    v = y[3:]
    dy[:3] = y[3:]
    # we dont use z'' = -g, but r'' = -G*r!, it's way more general
    dy[3:] = -G * r - 2 * cross(w, v)
    return dy

def derivative3(t, y, w):
    dy = np.zeros(6)
    r = y[:3]
    v = y[3:]
    dy[:3] = y[3:]
    # we dont use z'' = -g, but r'' = -G*r!, it's way more general
    dy[3:] = -G * r - 2 * cross(w, v) - cross(w, cross(w, r)) - ka * v * norm(v)
    return dy

d_fkts = [derivative1, derivative2, derivative3]

In [5]:
h0 = 500
times = np.linspace(0, np.sqrt(2 * h0 / g), 1000)
psi = 39 / 180 * np.pi
w = 7.27 * 1e-5 *np.array([-np.sin(psi), 0, np.cos(psi)])

Y0 = [0, 0, h0 + R0, 0, 0, 0]

In [6]:
print(norm(w)**2 * R0 * np.cos(psi)*np.sin(psi)/2, norm(w)*g/3*np.sin(psi))

0.008232896519296685 0.00014965345883631616


In [7]:
sol = solve_ivp(derivative1, [times[0], times[-1]], Y0, 
                t_eval=times, 
                args=(w,)
               )
sol.y[2] -=  R0

plt.close('first_test')
fig = plt.figure('first_test', figsize=(12, 6))
ax1 = fig.add_subplot(121, projection='3d')
ax2 = fig.add_subplot(122, projection='3d')

ax1.plot(sol.y[0], sol.y[1], sol.y[2])
ax2.plot(sol.t, sol.y[1], sol.y[2])

ax1.set_xlabel('x in m')
ax1.set_ylabel('y in m')
ax1.set_zlabel('z in m')

ax2.set_xlabel('t in s')
ax2.set_ylabel('x in m')
ax2.set_zlabel('z in m')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 0, 'z in m')

In [8]:
fig, ax = figure('sep_axis', 1, 3, figsize=(14, 4))

for i in range(3):
    ax[i].plot(sol.t, sol.y[i])
    ax[i].set_xlabel('t in s')


ax[0].plot(sol.t, norm(w)**2/2 * R0 * np.sin(psi) * np.cos(psi) * sol.t**2, 'k--')
ax[1].plot(sol.t, 9.813/3 * norm(w) * np.sin(psi) * sol.t**3, 'k--')
ax[2].plot(sol.t, -9.813/2 * sol.t**2 + Y0[2] - R0, 'k--')

ax[0].set_ylabel('x in m')
ax[1].set_ylabel('y in m')
ax[2].set_ylabel('z in m')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0, 0.5, 'z in m')

In [9]:
def get_slider():
    psi_s = FloatSlider(
        orientation='horizontal', description='$\psi$: ',
        value=psi/np.pi * 180, min=0, max=90, step = 1
    )
    w_s = FloatLogSlider(
        orientation='horizontal', description='$\omega$: ',
        value=norm(w), min=-6, max=2, step = 0.1, base=10
    )

    d_s = Select(
        options=['Full', 'without F_z', 'friction'],
        value='Full',
        rows=3,
        description='der: ',
        disabled=False
    )
    
    fig, ax = figure('dyn', 2, 3, figsize=(13, 5))
    lines = [ax[0, 0].plot(times, sol.y[0])[0],
             ax[0, 1].plot(times, sol.y[1])[0],
             ax[0, 2].plot(times, sol.y[2])[0],
             ax[1, 0].plot(times, sol.y[3])[0],
             ax[1, 1].plot(times, sol.y[4])[0],
             ax[1, 2].plot(times, sol.y[5])[0],
            ax[0, 1].plot(sol.t, 9.813/3 * w_s.value * np.sin(psi) * sol.t**3, 'k--')[0],
            ax[0, 2].plot(sol.t, -9.813/2 * sol.t**2 + Y0[2] - R0, 'k--')[0],
            ax[0, 0].plot(sol.t, norm(w)**2/2 * R0 * np.sin(psi) * np.cos(psi) * sol.t**2, 'k--')[0]]


    for i in range(3):
        ax[1, i].set_xlabel('t in s')
        
    ax[0, 0].set_ylabel('position')
    ax[1, 0].set_ylabel('velocity')
    for i, x in enumerate(['x', 'y', 'z']):
        ax[0, i].set_title(x)
    
    def redraw(change):
        psi = psi_s.value / 180 * np.pi 
        w = w_s.value
        sol = solve_ivp(d_fkts[d_s.index], [times[0], times[-1]], Y0, 
                t_eval=times, 
                args=(w_s.value * np.array([-np.sin(psi), 0, np.cos(psi)]),),
               )
        sol.y[2] -=  R0
        
        for i in range(6):
            lines[i].set_data(sol.t, sol.y[i])
            
        lines[6].set_data(sol.t, 9.813/3 * w * np.sin(psi) * sol.t**3)
        lines[7].set_data(sol.t, -9.813/2 * sol.t**2 + Y0[2] - R0)
        lines[8].set_data(sol.t, norm(w)**2/2 * R0 * np.sin(psi) * np.cos(psi) * sol.t**2)
        
        for i in range(3):
            for j in range(2):
                ax[j, i].relim()
                ax[j, i].autoscale_view()
            
        fig.canvas.draw()
        fig.canvas.flush_events()
        
    psi_s.observe(redraw, names='value')
    w_s.observe(redraw, names='value')
    d_s.observe(redraw, names='value')
    return HBox([psi_s, w_s, d_s])

get_slider()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## reverence frame

In [42]:
Tphi = lambda phi: np.array([[np.cos(phi),-np.sin(phi), 0],
                             [np.sin(phi), np.cos(phi), 0],
                             [          0,           0, 1]])
Tthe = lambda the: np.array([[ np.cos(the), 0, np.sin(the)],
                             [           0, 1,           0],
                             [-np.sin(the), 0, np.cos(the)]])

In [44]:
plt.close('test_rot')
fig = plt.figure('test_rot', figsize=(5, 5))
ax = fig.add_subplot(111, projection='3d')

v0 = np.array([0, 0, 0])
v1 = np.array([0, 0, 1])
v2 = Tthe(np.pi/4).dot(v1)
v3 = Tphi(np.pi/2).dot(v2)

ax.plot(*list(zip(v0, v1)))
ax.plot(*list(zip(v0, v2)))
ax.plot(*list(zip(v0, v3)))

ax.set_xlim([-1.1, 1.1])
ax.set_ylim([-1.1, 1.1])
ax.set_zlim([-1.1, 1.1])

ax.set_xlabel('x')
ax.set_ylabel('y')
ax.set_zlabel('z')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 0, 'z')

In [46]:
times2 = np.linspace(0, 60*60, 1000)
psi = 51 / 180 * np.pi
w = 7.27 * 1e-5 *np.array([-np.sin(psi), 0, np.cos(psi)])

def get_slider2():
    psi_s = FloatSlider(
        orientation='horizontal', description='$\psi$: ',
        value=psi/np.pi * 180, min=0, max=90, step = 1
    )
    
    w_s = FloatLogSlider(
        orientation='horizontal', description='$\omega$: ',
        value=norm(w), min=-6, max=2, step = 0.1, base=10
    )
    
    d_s = Select(
        options=['Full', 'without F_z', 'friction'],
        value='Full',
        rows=3,
        description='der: ',
        disabled=False
    )
    
    plt.close('ref_sol')
    fig = plt.figure('ref_sol', figsize=(6, 7))
    ax = fig.add_subplot(111, projection='3d')

    u, v = np.mgrid[0:2*np.pi:48j, 0:np.pi:48j]
    x = R0 * np.cos(u)*np.sin(v)
    y = R0 * np.sin(u)*np.sin(v)
    z = R0 * np.cos(v)
    # ax.plot_surface(x, y, z, color="k", alpha=0.15)
    ax.plot_wireframe(x, y, z, color="k", alpha=0.1, rstride=2, cstride=2)

    ax.set_xlabel('x in m')
    ax.set_ylabel('y in m')
    ax.set_zlabel('z in m')
    ax.set_box_aspect((1, 1, 1))

    Y0 = [0, 0, R0, 0, 0, 0]

    sol = solve_ivp(derivative1, [times2[0], times2[-1]], Y0, 
                    t_eval=times2, 
                    args=(w,),
                    rtol=1e-10,
                    atol=1e-12,
                    method='DOP853'
                   )

    ref_coords = np.zeros_like(sol.y[:3])
    w0 = norm(w)
    for i, t in enumerate(sol.t):
        ref_coords[:, i] = Tphi(w0 * t).dot(Tthe(psi).dot(sol.y[:3, i]))

    lines = [ax.plot(*ref_coords[:,0], 'ko')[0],
             ax.plot(*ref_coords)[0]]
    
    
    def redraw(change):
        psi = psi_s.value * np.pi/180
        w0 = w_s.value
        w = w0 * np.array([-np.sin(psi), 0, np.cos(psi)])
        sol = solve_ivp(d_fkts[d_s.index], [times2[0], times2[-1]], Y0, 
                t_eval=times2, 
                args=(w,),
                rtol=1e-10,
                atol=1e-12,
                method='DOP853'
               )

        ref_coords = np.zeros_like(sol.y[:3])
        for i, t in enumerate(sol.t):
            ref_coords[:, i] = Tphi(w0 * t).dot(Tthe(psi).dot(sol.y[:3, i]))
            
        lines[0].set_data(ref_coords[:2, 0])
        lines[0].set_3d_properties(ref_coords[2, 0])
        lines[1].set_data(ref_coords[:2, :])
        lines[1].set_3d_properties(ref_coords[2, :])
            
        fig.canvas.draw()
        fig.canvas.flush_events()
        
    psi_s.observe(redraw, names='value')
    w_s.observe(redraw, names='value')
    d_s.observe(redraw, names='value')
    return HBox([psi_s, w_s, d_s])

get_slider2()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Friction

In [47]:
v0 = 79.62579883802087
h0 = 500
ts = np.linspace(0, 10, 100)

vt = - np.sqrt(g / ka) * np.tan(np.sqrt(ka * g) * ts) + v0
vd = lambda t: -np.sqrt(g/ka) * np.tanh(np.sqrt(ka * g) * t)
zr = lambda t, h: h - 1/ka * np.log(np.cosh(np.sqrt(ka * g) * t))

print(1 / np.sqrt(ka * g) * np.arctan(np.sqrt(ka/g) * v0))
t0 = 1 / np.sqrt(ka * g) * np.arccosh(np.exp(h0*ka))
print(t0, vd(t0), np.sqrt(g/ka))

t1 = np.linspace(0, 1/np.sqrt(g*ka) * np.arccosh(np.exp(ka*h0)), 100)
z1 = zr(t1, h0)

ts = 1/np.sqrt(g*ka) * np.arctan(np.sqrt(ka/g) * abs(vd(t0)))
zs = - 1 / ka * np.log(np.cos(np.sqrt(g*ka) * ts))

t2 = np.linspace(0, abs(ts))

print(ts, zs)
vu = lambda t: np.sqrt(g/ka) * np.tan(np.sqrt(ka * g) * (t - ts))
zu = lambda t, h: h + 1/ka * np.log(np.cos(np.sqrt(ka * g) * (t - ts)))
print(zu(t2[-1], zs))

4.8763718568903895
14.1923340159109 -45.30315149516195 45.502505719580824
3.631681304377435 72.66253536321808
72.66253536321808


In [48]:
fig, ax = figure('frigger', 1, 2)
ax[0].plot(t1, z1)
ax[0].plot(t2 + t1[-1], zu(t2, zs))
ax[1].plot(t1, vd(t1))
ax[1].plot(t2 + t1[-1], vu(t2))
# ax[0].plot(t1 + t1[-1], -zu(t1, zs))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …